In [ ]:
import uproot
from matplotlib.colors import LogNorm
import matplotlib.colors as mcolors
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.constants import epsilon_0, e as q_e

import matplotlib as mpl
# Enable LaTeX rendering
mpl.rcParams['text.usetex'] = False
# Set the global font size
mpl.rcParams.update({'font.size': 12})

from matplotlib.colors import Normalize
from matplotlib.colorbar import ColorbarBase

import glob
import os

import trimesh
from trimesh.points import PointCloud

from common_functions import *

# from scipy.spatial import cKDTree
# import pyvista as pv
# pv.set_jupyter_backend('trame')  # or 'panel' if using panel

# Import results for runs with 1 million particles 

In [ ]:
# import geometry
stacked_spheres = trimesh.load_mesh('../sphere-charging/geometry/stacked_spheres_frompython_cropped.stl') 

In [ ]:
# Configuration: choose one
config = ["onlysolarwind"] #["onlysolarwind","onlyphotoemission"]  # or "onlysolarwind", "allparticles", "onlyphotoemission"

# Set selectnum 
selectnum = 100000 #10000000

# Define directory and file list
directory_path = "/storage/scratch1/5/avira7/Grain-Charging-Simulation-Data/stacked-sphere/output111025/dissipationRefinedGrid-initial8max0.8final12-past100/root/"
#directory_path = "/storage/scratch1/5/avira7/Grain-Charging-Simulation-Data/build-initial8max0.8final12-adjustedWorld-onlyPE/root/"
#directory_path = "../build-hcp-spheres/root/"
#directory_path = "../build-realistic-grains/root/"
filelist = sorted(glob.glob(f"{directory_path}/*iteration0*{config}*_num{selectnum}.root"))

# Process each ROOT file
for fileIN in filelist:
    filename = os.path.basename(fileIN)
    
    print(filename)

    # Extract iteration number
    number_str = filename.split("_")[1]
    iterationNUM = int(''.join(filter(str.isdigit, number_str)))

    # Extract config from filename (3rd part after split by "_")
    config_from_file = filename.split("_")[2]

    # Read data
    vars()["df_"+config_from_file] = read_rootfile(filename, directory_path=directory_path)
    #_ = calculate_stats(vars()["df_"+config_from_file], config=config_from_file)
    print("df_"+config_from_file)

    print("-" * 78)


# Check only SW case

In [ ]:
# check the input distribution for the electrons

fig, ax = plt.subplots(1,2,figsize=(5,4))

energy_values = np.linspace(0,50)
histdata = ax.hist(df_onlysolarwind[df_onlysolarwind["Particle_Type"] == "e-"].drop_duplicates(subset="Event_Number", keep="first")["Kinetic_Energy_Pre_MeV"]*1e6,bins=100,density=True)
electron_distribution = np.exp(-energy_values**2/10**2)*max(histdata[0][10:])
#plt.plot(energy_values, electron_distribution, "r-")
ax.set_xlim([-2,30])
ax.set_ylim([1e-5,0.12])
ax.set_xlabel(r"e$^-$ energy (eV)")
#ax[0].set_yscale("log")

energy_bins = np.linspace(1-0.0098,1+0.0098,50)
histdata = ax[1].hist(df_onlysolarwind[df_onlysolarwind["Particle_Type"] == "proton"].drop_duplicates(subset="Event_Number", keep="first")["Kinetic_Energy_Pre_MeV"]*1000,bins=energy_bins,density=True)
ax[1].set_xlabel(r"p$^+$ energy (keV)")

plt.show()

In [ ]:
fig, ax = plt.subplots(
    1, 2,
    figsize=(5, 4),
    gridspec_kw={'width_ratios': [8, 1]}  # ax[0] is 8× wider than ax[1]
)

energy_values = np.linspace(0, 50)
histdata = ax[0].hist(
    df_onlysolarwind[df_onlysolarwind["Particle_Type"] == "e-"]
    .drop_duplicates(subset="Event_Number", keep="first")["Kinetic_Energy_Pre_MeV"] * 1e6,
    bins=100,
    density=True
)

ax[0].set_xlim([0, 25])
ax[0].set_ylim([1e-5, 0.12])
ax[0].set_xlabel(r"e$^-$ energy (eV)")
ax[0].set_yticks([])

energy_bins = np.linspace(1 - 1e-2, 1 + 1e-2, 10)
histdata = ax[1].hist(
    df_onlysolarwind[df_onlysolarwind["Particle_Type"] == "proton"]
    .drop_duplicates(subset="Event_Number", keep="first")["Kinetic_Energy_Pre_MeV"] * 1000,
    bins=energy_bins,
    density=True
)

ax[1].set_yticks([])
ax[1].set_xticks([1])  # 👈 only show x=1
ax[1].set_xlabel(r"p$^+$(keV)")

plt.tight_layout()
plt.subplots_adjust(right=0.8,wspace=0.03)  # add a bit of space on the right
plt.show()



In [ ]:
proton_events = df_onlysolarwind[df_onlysolarwind["Particle_Type"]=="proton"]

# Extract positions into arrays
pre = np.vstack(proton_events["Pre_Step_Position_mm"].to_numpy())
post = np.vstack(proton_events["Post_Step_Position_mm"].to_numpy())

# Compute per-row distances (vectorized)
distances = np.linalg.norm(post - pre, axis=1)

# Add distances into dataframe
proton_events = proton_events.assign(Distance_mm = distances)

mask_SiO2 = (
    (proton_events["Volume_Name_Pre"] == "SiO2") &
    (proton_events["Volume_Name_Post"] == "SiO2")
)

# Keep only the SiO2-to-SiO2 steps
subset = proton_events[mask_SiO2]

# Sum distances per Event_Number
distance_per_event_nm = (
    subset.groupby("Event_Number")["Distance_mm"].sum() * 1e6  # mm → nm
)

plt.hist(distance_per_event_nm)
plt.show()

np.mean(distance_per_event_nm)


In [ ]:
df_onlysolarwind

In [ ]:
stacked_spheres = trimesh.load_mesh('../sphere-charging/geometry/stacked_spheres_frompython.stl') 

In [ ]:
# plot location of where all the elctrons & ion stopped in the geometry

last_protons = df_onlysolarwind[(df_onlysolarwind["Particle_Type"] == "proton")].drop_duplicates(subset="Event_Number", keep="last")
protons_inside = last_protons[(last_protons["Volume_Name_Post"] == "G4_SILICON_DIOXIDE")]
print(len(protons_inside))

last_electrons = df_onlysolarwind[(df_onlysolarwind["Particle_Type"] == "e-")].drop_duplicates(subset="Event_Number", keep="last")
electrons_inside = last_electrons[(last_electrons["Volume_Name_Post"] == "G4_SILICON_DIOXIDE")]
print(len(electrons_inside))

# Create a PointCloud object
proton_stopping_sites = PointCloud(np.array(protons_inside["Pre_Step_Position_mm"].tolist()), colors=[0, 255, 0, 255])  # RGBA red points
electron_stopping_sites = PointCloud(np.array(electrons_inside["Pre_Step_Position_mm"].tolist()), colors=[0, 0, 255, 255])  # RGBA red points

# Create a scene with both mesh and points
scene = trimesh.Scene([stacked_spheres, proton_stopping_sites,electron_stopping_sites])

# # apply this transformation to make the orientation match the Zimmerman paper
# T = np.array([
#     [-1, 0,  0, 0],
#     [ 0, 1,  0, 0],
#     [ 0, 0,  1, 0],
#     [ 0, 0,  0, 1]
# ])
# scene = trimesh.Scene()
# scene.add_geometry(stacked_spheres, transform=T)
# scene.add_geometry(proton_stopping_sites, transform=T)
# scene.add_geometry(electron_stopping_sites, transform=T)


# Show scene in interactive viewer
scene.show()

In [ ]:
# plot location of where all the ion stopped in the geometry

surface,potential = plot_surface_potential_fornegativepositive_charge(electrons_inside, protons_inside, stacked_spheres, vmin=-4,vmax=4)
print(min(potential), max(potential))
# Make sure each triangle has its own unique vertices
surface_edited = surface.copy()
surface_edited.unmerge_vertices()
surface_edited.visual.vertex_colors = None
surface_edited.show()

In [ ]:
# Define colormap and normalization
cmap = plt.cm.seismic
norm = Normalize(vmin=-1, vmax=1)

# Create a figure and a single axis for the colorbar
fig, ax = plt.subplots(figsize=(6, 1))
fig.subplots_adjust(bottom=0.5)

# Create the colorbar
cb = ColorbarBase(ax, cmap=cmap, norm=norm, orientation='horizontal')
cb.set_label('Surface Potential (mV)')  # Optional label

plt.show()


In [ ]:
surface,potential = plot_surface_potential_fornegativepositive_charge(electrons_inside, protons_inside, stacked_spheres, vmin=-1,vmax=1)
surface_edited = surface.copy()
surface_edited.unmerge_vertices()
surface_edited.visual.vertex_colors = None

bbox_min = np.array([-0.25, -0.31, 0])
bbox_max = np.array([ 0.15,  0.1,  100])

# Filter vertices
in_box = np.all((surface_edited.vertices >= bbox_min) & 
                (surface_edited.vertices <= bbox_max), axis=1)

# Get face indices where all 3 vertices are in the box
face_mask = np.all(in_box[surface_edited.faces], axis=1)

# Extract submesh
cropped = surface_edited.submesh([face_mask], only_watertight=False, append=True)
cropped.show()


In [ ]:
indicent_electrons = df_onlysolarwind[(df_onlysolarwind["Particle_Type"]=="e-") & (df_onlysolarwind["Parent_ID"]==0.0) &\
                (df_onlysolarwind["Volume_Name_Post"] == "G4_SILICON_DIOXIDE")
].drop_duplicates(subset="Event_Number", keep="first")

indicent_protons = df_onlysolarwind[(df_onlysolarwind["Particle_Type"]=="proton") & (df_onlysolarwind["Parent_ID"]==0.0) & \
                (df_onlysolarwind["Volume_Name_Post"] == "G4_SILICON_DIOXIDE")
].drop_duplicates(subset="Event_Number", keep="first")

merged_df = pd.concat([indicent_electrons, indicent_protons], ignore_index=True)

surface,face_totals = plot_face_illumination(merged_df, stacked_spheres, vmin=0,vmax=120)
print(min(face_totals), max(face_totals))
# Make sure each triangle has its own unique vertices
surface_edited = surface.copy()
surface_edited.unmerge_vertices()
surface_edited.show()

In [ ]:
bbox_min = np.array([-0.25, -0.31, 0])
bbox_max = np.array([ 0.15,  0.1,  100])

# Filter vertices
in_box = np.all((surface_edited.vertices >= bbox_min) & 
                (surface_edited.vertices <= bbox_max), axis=1)

# Get face indices where all 3 vertices are in the box
face_mask = np.all(in_box[surface_edited.faces], axis=1)

# Extract submesh
cropped = surface_edited.submesh([face_mask], only_watertight=False, append=True)
cropped.show()

# Check only Photoelectrons case

In [ ]:
fig, ax = plt.subplots(figsize=(5,3.2))

# check the input distribution for the photons
input_dist = pd.read_csv("../sphere-charging/distributions/photonSolar_distribution.txt",header=None,delimiter=" ")

incident_gamma = df_onlyphotoemission[(df_onlyphotoemission["Particle_Type"] == "gamma")&(df_onlyphotoemission["Parent_ID"] == 0.0)].drop_duplicates(subset="Event_Number", keep="first")
histdata = ax.hist(np.array(incident_gamma["Kinetic_Energy_Pre_MeV"]*1e6),bins=np.logspace(0.8,3.1,100),density=True,alpha=0.5,color='k')
print("# of incident gammas: ", len(incident_gamma))
plt.loglog(input_dist[0]*1e6,input_dist[1]/np.max(histdata[0])*2.0,'k-')
ax.set_xlim([6,330])
ax.set_ylim([1e-4,1])
ax.set_xlabel(r"Photon Energy (eV)")
#ax.axvline(x=1239.8/121.567)
ax.set_yscale("log")
ax.set_xscale("log")
#ax.set_yticklabels([])
ax.set_ylabel("Differential Flux")

plt.show()

In [ ]:
# ----------------------------
# First histogram: all photoelectrons
photoelectron_creation = df_onlyphotoemission[df_onlyphotoemission["Particle_Type"] == "e-"].drop_duplicates(subset="Event_Number", keep="first")
bins = np.logspace(-4, 3.2, 200)

# histogram counts
hist_all, bin_edges = np.histogram(photoelectron_creation["Kinetic_Energy_Pre_MeV"]*1e6, bins=bins, density=False)
bin_widths = np.diff(bin_edges)
bin_centers = 0.5*(bin_edges[1:] + bin_edges[:-1])

# ----------------------------
# Second histogram: only e- that actually leave
last_e_event = df_onlyphotoemission[df_onlyphotoemission["Particle_Type"] == "e-"].drop_duplicates(subset="Event_Number", keep="last")
world_e_energy = last_e_event[(last_e_event["Volume_Name_Post"]=="physical_cyclic") | 
                              (last_e_event["Volume_Name_Pre"]=="physical_cyclic")]

#matching_event_numbers = np.intersect1d(photoelectron_creation["Event_Number"], world_e_energy["Event_Number"])
#initial_PEenergy_leading_toejection = photoelectron_creation[photoelectron_creation["Event_Number"].isin(matching_event_numbers)]

Zmin, Zmax = np.min(np.vstack(world_e_energy["Post_Step_Position_mm"])[:,2]), np.max(np.vstack(world_e_energy["Post_Step_Position_mm"])[:,2])
ind = np.argwhere((np.vstack(world_e_energy["Post_Step_Position_mm"])[:,2]>=Zmax*0.99) |
                  (np.vstack(world_e_energy["Post_Step_Position_mm"])[:,2]<=Zmin*0.99))
plt.plot(np.vstack(world_e_energy["Post_Step_Position_mm"])[:,0], np.vstack(world_e_energy["Post_Step_Position_mm"])[:,2], 'k.',markersize=0.1)
plt.plot(np.vstack(world_e_energy["Post_Step_Position_mm"])[:,0][ind], np.vstack(world_e_energy["Post_Step_Position_mm"])[:,2][ind], 'b.',markersize=0.1)
plt.show()

# histogram counts
counts_ejected, _ = np.histogram(np.vstack(world_e_energy["Kinetic_Energy_Post_MeV"])[ind]*1e6, bins=bins, density=False)

# ----------------------------
# Scaling: scale the ejected counts to match total counts of all photoelectrons
total_all = np.sum(hist_all)
total_ejected = np.sum(counts_ejected)
precent_ejected = total_ejected/total_all

counts_ejected_scaled = counts_ejected

# ----------------------------
# Convert both to differential flux (counts per bin width)
differential_allPE = hist_all / bin_widths
differential_ejected_scaled = counts_ejected_scaled / bin_widths

# Filter out zero/very small values to avoid isolated bars
# Set a threshold based on the minimum reasonable value
threshold = 0.1  # adjust as needed
mask_all = differential_allPE > threshold
mask_ejected = differential_ejected_scaled > threshold

# Apply mask and replace filtered values with NaN for cleaner plotting
diff_all_filtered = np.where(mask_all, differential_allPE, np.nan)
diff_ejected_filtered = np.where(mask_ejected, differential_ejected_scaled, np.nan)

fig, ax = plt.subplots(figsize=(5,3.2))

# Plot - using linewidth=0 removes the vertical lines between steps
ax.step(bin_centers, diff_all_filtered, where='mid', color='darkblue', alpha=0.7, linewidth=1.5, label='All photoelectrons')
ax.fill_between(bin_centers, diff_all_filtered, step='mid', color='darkblue', alpha=0.3, linewidth=0)

ax.step(bin_centers, diff_ejected_filtered, where='mid', color='darkgreen', alpha=0.7, linewidth=1.5, label='Emitted e-')
ax.fill_between(bin_centers, diff_ejected_filtered, step='mid', color='darkgreen', alpha=0.3, linewidth=0)

# ----------------------------
ax.set_xlabel("Photoelectron Energy (eV)")
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlim([bins[0], 330])
ax.set_ylabel("Differential Flux")
#ax.set_yticklabels([])
ax.set_ylim(10)

print("Average photoelectron energy: ", np.mean(photoelectron_creation["Kinetic_Energy_Pre_MeV"]*1e6), " eV")
print("# of photoelectrons: ", len(photoelectron_creation))
print(f"# of emitted e-, reaching the top surface: {len(ind)}")
print(f"% emitted e-: {len(ind)/len(photoelectron_creation)*100:.5f}")

plt.show()

In [ ]:
df_e = df_onlyphotoemission[df_onlyphotoemission["Particle_Type"] == "e-"]

In [ ]:
df_e[df_e["Volume_Name_Pre"]=="physical_cyclic"][0:50]

In [ ]:
df_onlyphotoemission[df_onlyphotoemission["Event_Number"] ==99633]

In [ ]:
# Section of geometry
section = stacked_spheres.section(
    plane_origin=stacked_spheres.centroid,
    plane_normal=[0, 1, 0]
)
plane_point = stacked_spheres.centroid
normal = np.array([0,1,0], float)

# project vertices manually
verts = section.vertices

# project into plane (XY plane mapped as X,Z)
V = verts - plane_point      # center
proj_x = V[:,0]              # global X
proj_y = V[:,2]              # global Z

# Setup plot
fig, ax = plt.subplots(figsize=(8,7))

from shapely.geometry import Polygon as ShapelyPolygon, Point
from shapely.ops import unary_union
from matplotlib.patches import Polygon
import numpy as np

# Collect polygons and identify them
sphere_sections = []
gap_sections = []

for e in section.entities:
    pts = np.array([[proj_x[p], proj_y[p]] for p in e.points])
    
    if len(pts) < 3:
        continue
    
    # Close the polygon if needed
    if not np.allclose(pts[0], pts[-1]):
        pts = np.vstack([pts, pts[0:1]])
    
    try:
        poly = ShapelyPolygon(pts)
        
        if not poly.is_valid:
            poly = poly.buffer(0)  # Fix invalid geometries
        
        if poly.is_empty or poly.area < 1e-8:
            continue
        
        # Calculate circularity: 4π*area/perimeter²
        # Perfect circle = 1, elongated shapes < 1
        circularity = 4 * np.pi * poly.area / (poly.length ** 2) if poly.length > 0 else 0
        
        # Sphere cross-sections should be roughly circular
        # Gaps between spheres will be elongated (low circularity)
        if circularity > 0.7:  # Adjust threshold as needed (0.7-0.9 typical)
            sphere_sections.append(poly)
        else:
            gap_sections.append(poly)
            print(f"Gap detected: area={poly.area:.6f}, circularity={circularity:.3f}")
            
    except Exception as e:
        print(f"Error processing entity: {e}")
        continue

# Fill only the sphere sections
print(f"Found {len(sphere_sections)} sphere sections, {len(gap_sections)} gaps")

for poly in sphere_sections:
    coords = np.array(poly.exterior.coords)
    patch = Polygon(coords, closed=True, facecolor='lightgray', 
                   edgecolor='none', alpha=0.4)
    ax.add_patch(patch)

# Optional: visualize gaps for debugging
# for poly in gap_sections:
#     coords = np.array(poly.exterior.coords)
#     patch = Polygon(coords, closed=True, facecolor='red', 
#                    edgecolor='none', alpha=0.2)
#     ax.add_patch(patch)

for e in section.entities:
    pts = e.points
    ax.plot(proj_x[pts], proj_y[pts], 'k-',lw=2,alpha=0.5)

for eventIN in [2543]: #99633, world_e_energy["Event_Number"][0:10]:
    # plot different particles as diffent colors
    eventdf = df_onlyphotoemission[df_onlyphotoemission["Event_Number"]==eventIN]
    gamma_eventdf = eventdf[eventdf["Particle_Type"] == "gamma"]
    e_eventdf = eventdf[eventdf["Particle_Type"] == "e-"]


    # Correct physical interleaving
    gamma_interwoven = np.empty((np.vstack(gamma_eventdf["Pre_Step_Position_mm"]).shape[0] + np.vstack(gamma_eventdf["Post_Step_Position_mm"]).shape[0], 3))
    gamma_interwoven[0::2] = np.vstack(gamma_eventdf["Pre_Step_Position_mm"])
    gamma_interwoven[1::2] = np.vstack(gamma_eventdf["Post_Step_Position_mm"])

    # Correct physical interleaving
    e_interwoven = np.empty((np.vstack(e_eventdf["Pre_Step_Position_mm"]).shape[0] + np.vstack(e_eventdf["Post_Step_Position_mm"]).shape[0], 3))
    e_interwoven[0::2] = np.vstack(e_eventdf["Pre_Step_Position_mm"])
    e_interwoven[1::2] = np.vstack(e_eventdf["Post_Step_Position_mm"])

    gamma_interwoven = np.unique(gamma_interwoven,axis=0)
    e_interwoven = np.unique(e_interwoven,axis=0)

    ax.plot(gamma_interwoven[:, 1], gamma_interwoven[:,2], '-', markersize=6, color='blue', alpha=0.8,lw=4)
    ax.plot(e_interwoven[:, 1], e_interwoven[:,2], '-', markersize=6, color='red',alpha=0.8,lw=2)
ax.set_ylim([-0.2+0.030,0.203])
ax.set_xlim([-0.2,0.2])
ax.axis("off")
#plt.savefig("figures/PE_event_demo.svg",transparent=True)
plt.show()


In [ ]:
# Section of geometry
section = stacked_spheres.section(
    plane_origin=stacked_spheres.centroid,
    plane_normal=[0, 1, 0]
)
plane_point = stacked_spheres.centroid
normal = np.array([0,1,0], float)

# project vertices manually
verts = section.vertices

# project into plane (XY plane mapped as X,Z)
V = verts - plane_point      # center
proj_x = V[:,0]              # global X
proj_y = V[:,2]              # global Z

# Setup plot
fig, ax = plt.subplots(figsize=(8,7))
from shapely.geometry import Polygon as ShapelyPolygon
from shapely.ops import unary_union
from shapely.validation import make_valid
from matplotlib.patches import Polygon
import numpy as np

# Convert to shapely polygons with filtering
shapely_polys = []
for e in section.entities:
    pts = np.array([[proj_x[p], proj_y[p]] for p in e.points])
    
    # Skip if too few points
    if len(pts) < 3:
        continue
    
    # Close the loop if needed
    if not np.allclose(pts[0], pts[-1]):
        pts = np.vstack([pts, pts[0:1]])
    
    try:
        poly = ShapelyPolygon(pts)
        
        # Only keep valid polygons that represent actual sphere cross-sections
        # Filter out small gaps/artifacts by area threshold
        if poly.is_valid and poly.area > 1e-6:  # Adjust threshold as needed
            # Optional: check if polygon is roughly circular (sphere section)
            # by comparing area to bounding circle
            shapely_polys.append(poly)
    except:
        continue

# Union all valid polygons
if shapely_polys:
    try:
        unified = unary_union(shapely_polys)
        
        # Handle both single polygon and multipolygon results
        if unified.geom_type == 'Polygon':
            unified = [unified]
        elif unified.geom_type == 'MultiPolygon':
            unified = list(unified.geoms)
        else:
            unified = []
        
        # Plot unified regions, filtering out small gaps
        for poly in unified:
            # Skip small artifacts (gaps between spheres)
            if poly.area < 1e-5:  # Adjust threshold
                continue
                
            coords = np.array(poly.exterior.coords)
            patch = Polygon(coords, closed=True, facecolor='lightgray', 
                           edgecolor='none', alpha=0.4)
            ax.add_patch(patch)
    except Exception as e:
        print(f"Union failed: {e}")
for e in section.entities:
    pts = e.points
    ax.plot(proj_x[pts], proj_y[pts], 'k-',lw=2,alpha=0.5)

for eventIN in [2543]: #99633, world_e_energy["Event_Number"][0:10]:
    # plot different particles as diffent colors
    eventdf = df_onlyphotoemission[df_onlyphotoemission["Event_Number"]==eventIN]
    gamma_eventdf = eventdf[eventdf["Particle_Type"] == "gamma"]
    e_eventdf = eventdf[eventdf["Particle_Type"] == "e-"]


    # Correct physical interleaving
    gamma_interwoven = np.empty((np.vstack(gamma_eventdf["Pre_Step_Position_mm"]).shape[0] + np.vstack(gamma_eventdf["Post_Step_Position_mm"]).shape[0], 3))
    gamma_interwoven[0::2] = np.vstack(gamma_eventdf["Pre_Step_Position_mm"])
    gamma_interwoven[1::2] = np.vstack(gamma_eventdf["Post_Step_Position_mm"])

    # Correct physical interleaving
    e_interwoven = np.empty((np.vstack(e_eventdf["Pre_Step_Position_mm"]).shape[0] + np.vstack(e_eventdf["Post_Step_Position_mm"]).shape[0], 3))
    e_interwoven[0::2] = np.vstack(e_eventdf["Pre_Step_Position_mm"])
    e_interwoven[1::2] = np.vstack(e_eventdf["Post_Step_Position_mm"])

    gamma_interwoven = np.unique(gamma_interwoven,axis=0)
    e_interwoven = np.unique(e_interwoven,axis=0)

    ax.plot(gamma_interwoven[:, 1], gamma_interwoven[:,2], '-', markersize=6, color='blue', alpha=0.8,lw=4)
    ax.plot(e_interwoven[:, 1], e_interwoven[:,2], '-', markersize=6, color='red',alpha=0.8,lw=2)
ax.set_ylim([-0.2+0.030,0.203])
ax.set_xlim([-0.2,0.2])
ax.axis("off")
#plt.savefig("figures/PE_event_demo.svg",transparent=True)
plt.show()


In [ ]:
labels

In [ ]:
e_interwoven

In [ ]:
np.unique(e_interwoven,axis=0)

In [ ]:
fig4data = pd.read_csv("literature-data/Fig4-Feurerbacher1972.csv")
fig3data = pd.read_csv("literature-data/Fig3Feuerbacher.csv")

In [ ]:
# ----------------------------

bins = np.logspace(0.8, 3.4, 200)

# histogram counts
incident_gamma = df_onlyphotoemission[(df_onlyphotoemission["Particle_Type"] == "gamma")&(df_onlyphotoemission["Parent_ID"] == 0.0)].drop_duplicates(subset="Event_Number", keep="first")
hist_all, bin_edges = np.histogram(incident_gamma["Kinetic_Energy_Pre_MeV"]*1e6,bins=bins)

bin_widths = np.diff(bin_edges)
bin_centers = 0.5*(bin_edges[1:] + bin_edges[:-1])

# ----------------------------
# Second histogram: only e- that actually leave
last_e_event = df_onlyphotoemission[df_onlyphotoemission["Particle_Type"] == "e-"].drop_duplicates(subset=["Parent_ID", "Event_Number"], keep="last")
#df_onlyphotoemission[df_onlyphotoemission["Particle_Type"] == "e-"].drop_duplicates(subset="Event_Number", keep="last")
world_e_energy = last_e_event[(last_e_event["Volume_Name_Post"]=="physical_cyclic") | 
                              (last_e_event["Volume_Name_Pre"]=="physical_cyclic")]

# matching_event_numbers = np.intersect1d(incident_gamma["Event_Number"], world_e_energy["Event_Number"])
# initial_PEenergy_leading_toejection = incident_gamma[incident_gamma["Event_Number"].isin(matching_event_numbers)]

# histogram counts
# counts_ejected, _ = np.histogram(initial_PEenergy_leading_toejection["Kinetic_Energy_Pre_MeV"]*1e6, bins=bins, density=False)

Zmin, Zmax = np.min(np.vstack(world_e_energy["Post_Step_Position_mm"])[:,2]), np.max(np.vstack(world_e_energy["Post_Step_Position_mm"])[:,2])
ind = np.argwhere((np.vstack(world_e_energy["Post_Step_Position_mm"])[:,2]>=Zmax*0.99) |
                  (np.vstack(world_e_energy["Post_Step_Position_mm"])[:,2]<=Zmin*0.99))
plt.plot(np.vstack(world_e_energy["Post_Step_Position_mm"])[:,0], np.vstack(world_e_energy["Post_Step_Position_mm"])[:,2], 'k.',markersize=0.1)
plt.plot(np.vstack(world_e_energy["Post_Step_Position_mm"])[:,0][ind], np.vstack(world_e_energy["Post_Step_Position_mm"])[:,2][ind], 'b.',markersize=0.1)
plt.show()

# histogram counts
counts_ejected, _ = np.histogram(np.vstack(world_e_energy["Kinetic_Energy_Post_MeV"])[ind]*1e6, bins=bins, density=False)

# ----------------------------
# Calculate emission efficiency (% yield) per bin
# Avoid division by zero
emission_efficiency = np.divide(counts_ejected, hist_all, 
                                where=hist_all>0, 
                                out=np.zeros_like(counts_ejected, dtype=float))
emission_percent = emission_efficiency * 100

#fig, ax = plt.subplots(figsize=(9.5,3.5))
fig, ax = plt.subplots(figsize=(5,3.2))

# Plot
ax.plot(bin_centers, emission_efficiency, color='darkgreen', linewidth=1.5, label='Emission yield')
ax.fill_between(bin_centers, emission_efficiency, color='darkgreen', alpha=0.3, linewidth=0)

# Add literature data
ax.plot(1239.8/(fig3data["x"]/10), fig3data[" y"], 'k-',label="Literature")
# Kimura_a10nm = pd.read_csv("literature-data/Fig2aKimura_a=10nmSolid.csv")
# Kimura_bulk = pd.read_csv("literature-data/Fig2aKimura_bulk.csv")
# ax.plot(Kimura_a10nm["x"], Kimura_a10nm[" y"], linestyle='-.',color="blue",label="Literature")
# ax.plot(Kimura_bulk["x"], Kimura_bulk[" y"], linestyle='-.',color="orange", label="Literature")

# ----------------------------
ax.set_xlabel("Photon Energy (eV)")
ax.set_xscale("log")
ax.set_xlim([5, 350])
ax.set_ylabel("Yield Per Inc. Photon")
ax.set_ylim([1e-3, 1])
ax.set_yscale("log")
ax.set_xscale("log")

print("Average photoelectron energy: ", np.mean(photoelectron_creation["Kinetic_Energy_Pre_MeV"]*1e6), " eV")
print("# of photoelectrons: ", len(photoelectron_creation))
print(f"# of emitted e-, reaching top surface: {len(ind)}")
print(f"Overall ejection yield: {len(ind)/len(incident_gamma)*100:.3f}%")

plt.show()

In [ ]:
df_onlyphotoemission

In [ ]:
# ----------------------------
# Define energy bins
bins = np.logspace(0.8, 3.3, 100)

# Get photoelectron creation events (initial PE energy at creation)
photoelectron_creation = df_onlyphotoemission[df_onlyphotoemission["Particle_Type"] == "e-"].drop_duplicates(subset="Event_Number", keep="first")

bin_edges = bins
bin_widths = np.diff(bin_edges)
bin_centers = 0.5*(bin_edges[1:] + bin_edges[:-1])

# ----------------------------
# Get only electrons that actually leave the material
last_e_event = df_onlyphotoemission[df_onlyphotoemission["Particle_Type"] == "e-"].drop_duplicates(subset=["Parent_ID", "Event_Number"], keep="last")
world_e_energy = last_e_event[(last_e_event["Volume_Name_Post"]=="physical_cyclic") | 
                              (last_e_event["Volume_Name_Pre"]=="physical_cyclic")]

# # Find which photoelectron creation events led to ejection
# matching_event_numbers = np.intersect1d(photoelectron_creation["Event_Number"], world_e_energy["Event_Number"])
# initial_PEenergy_leading_toejection = photoelectron_creation[photoelectron_creation["Event_Number"].isin(matching_event_numbers)]
# # - Ejected electrons (binned by the initial photon energy that created them)
# counts_ejected, _ = np.histogram(initial_PEenergy_leading_toejection["Kinetic_Energy_Pre_MeV"]*1e6, bins=bins)

# Histogram counts: 
# - Total absorption events (photoelectric effect) at each photon energy
hist_absorption_events, _ = np.histogram(photoelectron_creation["Kinetic_Energy_Pre_MeV"]*1e6, bins=bins)

Zmin, Zmax = np.min(np.vstack(world_e_energy["Post_Step_Position_mm"])[:,2]), np.max(np.vstack(world_e_energy["Post_Step_Position_mm"])[:,2])
ind = np.argwhere((np.vstack(world_e_energy["Post_Step_Position_mm"])[:,2]>=Zmax*0.99) |
                  (np.vstack(world_e_energy["Post_Step_Position_mm"])[:,2]<=Zmin*0.99))
plt.plot(np.vstack(world_e_energy["Post_Step_Position_mm"])[:,0], np.vstack(world_e_energy["Post_Step_Position_mm"])[:,2], 'k.',markersize=0.1)
plt.plot(np.vstack(world_e_energy["Post_Step_Position_mm"])[:,0][ind], np.vstack(world_e_energy["Post_Step_Position_mm"])[:,2][ind], 'b.',markersize=0.1)
plt.show()

# histogram counts
counts_ejected, _ = np.histogram(np.vstack(world_e_energy["Kinetic_Energy_Post_MeV"])[ind]*1e6, bins=bins, density=False)

# ----------------------------
# Calculate yield per absorption event
# This is the escape probability: P(escape | absorption at energy E)
# This is ALREADY independent of input distribution - it's just a conditional probability
yield_per_absorption = np.divide(counts_ejected, hist_absorption_events, 
                                 where=hist_absorption_events>0, 
                                 out=np.zeros_like(counts_ejected, dtype=float))

# ----------------------------

#fig, ax = plt.subplots(figsize=(9.5,3.5))
fig, ax = plt.subplots(figsize=(5,3.2))

# Plot
ax.plot(bin_centers, yield_per_absorption, color='darkgreen', linewidth=1.5, label='Emission yield (simulation)')
ax.fill_between(bin_centers, yield_per_absorption, color='darkgreen', alpha=0.3, linewidth=0)

# Add literature data
# ax.plot(1240/(fig3data["x"]/10), fig3data[" y"], 'k-', label="Literature (Ref 1)")
Kimura_a10nm = pd.read_csv("literature-data/Fig2aKimura_a=10nmSolid.csv")
Kimura_bulk = pd.read_csv("literature-data/Fig2aKimura_bulk.csv")
ax.plot(Kimura_a10nm["x"], Kimura_a10nm[" y"], linestyle='-.', color="blue", label="Literature (a=10nm)")
ax.plot(Kimura_bulk["x"], Kimura_bulk[" y"], linestyle='-.', color="orange", label="Literature (bulk)")

# ----------------------------
ax.set_xlabel("Photon Energy (eV)")
ax.set_ylabel("Yield Per PE Event")
ax.set_xlim([5, 340])
ax.set_ylim([1e-3, 1])
ax.set_xscale("log")
ax.set_yscale("log")

plt.show()

In [ ]:
surface,face_totals,_ = plot_face_illumination(world_e_energy, stacked_spheres, vmin=0,vmax=3)
print(min(face_totals), max(face_totals))
# Make sure each triangle has its own unique vertices
surface_edited = surface.copy()
surface_edited.unmerge_vertices()
#surface_edited.visual.vertex_colors = None
surface_edited.show()

In [ ]:
# -----------------------------------------
# Step 0: Load field data and filter
# -----------------------------------------

geometry = stacked_spheres
positions = np.vstack(world_e_energy["Post_Step_Position_mm"])
 
geometry_center = geometry.centroid

# ----------------------------------------------------
# Step 1: Create FULL geometry (before cropping)
# ----------------------------------------------------
 
pv_spheres_full = pv.PolyData(
    geometry.vertices,
    np.hstack([np.full((len(geometry.faces), 1), 3), geometry.faces])
).compute_normals()

print(f"Full geometry has {pv_spheres_full.n_cells} faces")

# ============================================================
# Step 2: Calculate surface charge on FULL geometry
# ============================================================
num_faces_full = pv_spheres_full.n_cells
face_charges_full,face_counts_negative,face_counts_positive = np.zeros(num_faces_full),np.zeros(num_faces_full),np.zeros(num_faces_full)

# Face areas (m^2) for full geometry
face_areas_m2_full = pv_spheres_full.compute_cell_sizes()['Area'] * 1e-6 + 1e-20

# Get face centers for full geometry
face_centers_full = pv_spheres_full.cell_centers().points

# Build KDTree for fast nearest neighbor search
tree = cKDTree(face_centers_full)

# --- Bin protons to closest face ---
if len(positions) > 0:
    _, face_id_p = tree.query(positions, k=1)
    unique_faces, counts = np.unique(face_id_p, return_counts=True)
    face_counts_positive[unique_faces] += counts
    print(f"Binned {len(positions)} protons to {len(unique_faces)} unique faces")

pv_spheres_full.cell_data['positive_illumination'] = face_counts_positive

In [ ]:
# ----------------------------------------------------
# Visualization
# ----------------------------------------------------
pl = pv.Plotter()
pl.set_background('white')

pl.add_mesh(
    pv_spheres_full,
    # scalars='F_x', 
    # cmap="seismic",
    # clim=[vmin, vmax],

    scalars='positive_illumination', 
    cmap = "OrRd",
    clim=[0,10],
    opacity=1,
    show_edges=False,
    interpolate_before_map=False,
    preference="cell"
)

pl.view_xy()
pl.show(jupyter_backend='static')

In [ ]:
gamma_holes0, electrons_ejected0, electron_inside0 = calculate_stats(df_onlyphotoemission)

# plot location of where all the ion stopped in the geometry

# Create a PointCloud object
gamma_photoemission_sites = PointCloud(np.array(gamma_holes0["Pre_Step_Position_mm"].tolist()), colors=[0, 255, 0, 255])  # RGBA red points
electron_stopping_sites = PointCloud(np.array(electron_inside0["Pre_Step_Position_mm"].tolist()), colors=[0, 0, 255, 255])  # RGBA red points

# Create a scene with both mesh and points
stacked_spheres = trimesh.load_mesh('../sphere-charging/geometry/stacked_spheres_frompython.stl') 
scene = trimesh.Scene([stacked_spheres, gamma_photoemission_sites,electron_stopping_sites])

# Show scene in interactive viewer
scene.show()

In [ ]:
# plot the surface potential of the photoemission

surface,potential = plot_surface_potential_fornegativepositive_charge(electron_inside0, gamma_holes0, stacked_spheres, vmin=-0.2,vmax=0.2)

print(min(potential), max(potential))
plt.plot(potential[potential!=0])
plt.show()

# Make sure each triangle has its own unique vertices
surface_edited = surface.copy()
surface_edited.unmerge_vertices()
surface_edited.visual.vertex_colors = None
surface_edited.show()

# a lot of the surface charge ended up cancelling each other out...

In [ ]:
bbox_min = np.array([-0.25, -0.31, 0])
bbox_max = np.array([ 0.15,  0.1,  100])

# Filter vertices
in_box = np.all((surface_edited.vertices >= bbox_min) & 
                (surface_edited.vertices <= bbox_max), axis=1)

# Get face indices where all 3 vertices are in the box
face_mask = np.all(in_box[surface_edited.faces], axis=1)

# Extract submesh
cropped = surface_edited.submesh([face_mask], only_watertight=False, append=True)
cropped.show()


In [ ]:
# Define colormap and normalization
cmap = plt.cm.seismic
norm = Normalize(vmin=-0.2, vmax=0.2)

# Create a figure and a single axis for the colorbar
fig, ax = plt.subplots(figsize=(6, 1))
fig.subplots_adjust(bottom=0.5)

# Create the colorbar
cb = ColorbarBase(ax, cmap=cmap, norm=norm, orientation='horizontal')
cb.set_label('Surface Potential (mV)')  # Optional label

plt.show()


In [ ]:
indicent_gamma = df_onlyphotoemission[(df_onlyphotoemission["Particle_Type"]=="gamma") & (df_onlyphotoemission["Parent_ID"]==0.0) & \
                (df_onlyphotoemission["Volume_Name_Post"] == "G4_SILICON_DIOXIDE")
].drop_duplicates(subset="Event_Number", keep="first")

merged_df = pd.concat([indicent_gamma], ignore_index=True)

surface,face_totals = plot_face_illumination(merged_df, stacked_spheres, vmin=0,vmax=300)
print(min(face_totals), max(face_totals))
# Make sure each triangle has its own unique vertices
surface_edited = surface.copy()
surface_edited.unmerge_vertices()
#surface_edited.visual.vertex_colors = None
surface_edited.show()

In [ ]:
# where the photoelectrons are generated 
generated_PE = df_onlyphotoemission[(df_onlyphotoemission["Particle_Type"]=="e-") & (df_onlyphotoemission["Parent_ID"]==1.0) & \
                (df_onlyphotoemission["Process_Name_Pre"] == "initStep")
].drop_duplicates(subset="Event_Number", keep="first")

merged_df = pd.concat([generated_PE], ignore_index=True)

surface,face_totals = plot_face_illumination(merged_df, stacked_spheres, vmin=0,vmax=300)
print(min(face_totals), max(face_totals))
# Make sure each triangle has its own unique vertices
surface_edited = surface.copy()
surface_edited.unmerge_vertices()
#surface_edited.visual.vertex_colors = None
surface_edited.show()